## Outline
#### 1. Data Scraping- Cartalk
- Due to the large volume of data, the data scraping was divided into 12 separate sessions
- The first-try data scraping, there's a mistake for not scraping "Date"
- Therefore some of the code here are based on the dataset I've scraped, and the re-scraped "Date" are added into my previous dataset
#### 2. Data Labelling
- We don't use it in final project, We do manual labeling at the end
#### 3. Name Recognition 
#### 4. Sentiment Analaysis
#### 5. Topic Modelling 
#### 6. Time Series

### 1. Data scraping- Cartalk

In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# Use webdriver_manager to automatically download and manage chromedriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Set explicit wait time to 30 seconds
wait = WebDriverWait(driver, 30)

# Read topic links from the CSV file
topic_links = []
with open('cartalk_topic_links.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        topic_links.append(row[0])

# Only process the first 2000 links
topic_links = topic_links[:2000]

# List to store the scraped replies
replies_data = []

for link in topic_links:
    # Open each topic link
    driver.get(link)
    
    # Wait for the page to load
    time.sleep(8)
    
    # Get the page content
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    
    # Find the reply content and date
    replies = soup.find_all('div', class_='cooked')  # Assuming reply content is in a div with this class name
    dates = soup.find_all('span', class_='relative-date')  # Assuming dates are in a span with this class name
    
    for reply, date in zip(replies, dates):
        reply_text = reply.get_text(strip=True)
        reply_date = date['title'] if 'title' in date.attrs else date.get_text(strip=True)
        replies_data.append([link, reply_text, reply_date])
    
    print(f"Processed {link}")

# Save the scraped replies to a new CSV file
replies_csv_file = 'cartalk_topic_replies00.csv'
with open(replies_csv_file, mode='w', newline='', encoding='utf-8') as file: 
    writer = csv.writer(file)
    writer.writerow(['Topic Link', 'Reply', 'Date'])
    
    for data in replies_data:
        writer.writerow(data)

print(f"All replies have been saved to '{replies_csv_file}'")

# Close the browser
driver.quit()

In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# Use webdriver_manager to automatically download and manage chromedriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Set explicit wait time to 30 seconds
replies_file = 'cartalk_topic_replies1.csv'

# Read the topic links from the existing replies file
topic_links = []
with open(replies_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        topic_links.append(row[0])

# Only process the first 1000 links (adjust as needed)
topic_links = topic_links[:1000]

# List to store the scraped dates
dates_data = []

for link in topic_links:
    # Open each topic link
    driver.get(link)
    
    # Wait for the page to load
    time.sleep(8)
    
    # Get the page content
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    
    # Find the dates
    dates = soup.find_all('span', class_='relative-date')  # Assuming dates are in a span with this class name
    
    for date in dates:
        reply_date = date['title'] if 'title' in date.attrs else date.get_text(strip=True)
        dates_data.append([link, reply_date])
    
    print(f"Processed {link}")

# Save the scraped dates to a new CSV file
dates_csv_file = 'cartalk_topic_dates.csv'
with open(dates_csv_file, mode='w', newline='', encoding='utf-8') as file: 
    writer = csv.writer(file)
    writer.writerow(['Topic Link', 'Date'])
    
    for data in dates_data:
        writer.writerow(data)

print(f"All dates have been saved to '{dates_csv_file}'")

# Close the browser
driver.quit()

# Merge files
merged_file = 'cartalk_topic_replies01.csv'

# Read the existing replies file
replies_data = []
with open(replies_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)  # Read the header row
    for row in reader:
        replies_data.append(row)

# Read the dates file and create a dictionary
dates_dict = {}
with open(dates_csv_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        topic_link = row[0]
        reply_date = row[1]
        if topic_link in dates_dict:
            dates_dict[topic_link].append(reply_date)
        else:
            dates_dict[topic_link] = [reply_date]

# Merge the data
merged_data = []
for row in replies_data:
    topic_link = row[0]
    reply = row[1]
    date = dates_dict.get(topic_link, [''])[0]  # Default to an empty string if the date doesn't exist
    merged_data.append([topic_link, reply, date])
    if topic_link in dates_dict:
        dates_dict[topic_link].pop(0)  # Remove the used date

# Save the merged data
with open(merged_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Topic Link', 'Reply', 'Date'])
    
    for data in merged_data:
        writer.writerow(data)

print(f"All data have been merged and saved to '{merged_file}'")



In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# Use webdriver_manager to automatically download and manage chromedriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Set explicit wait time to 30 seconds
replies_file = 'cartalk_topic_replies2.csv'

# Read the topic links from the existing replies file
topic_links = []
with open(replies_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        topic_links.append(row[0])

# Only process the first 2000 links (adjust as needed)
topic_links = topic_links[:2000]

# List to store the scraped dates
dates_data = []

for link in topic_links:
    # Open each topic link
    driver.get(link)
    
    # Wait for the page to load
    time.sleep(8)
    
    # Get the page content
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    
    # Find the dates
    dates = soup.find_all('span', class_='relative-date')  # Assuming dates are in a span with this class name
    
    for date in dates:
        reply_date = date['title'] if 'title' in date.attrs else date.get_text(strip=True)
        dates_data.append([link, reply_date])
    
    print(f"Processed {link}")

# Save the scraped dates to a new CSV file
dates_csv_file = 'cartalk_topic_dates2.csv'
with open(dates_csv_file, mode='w', newline='', encoding='utf-8') as file: 
    writer = csv.writer(file)
    writer.writerow(['Topic Link', 'Date'])
    
    for data in dates_data:
        writer.writerow(data)

print(f"All dates have been saved to '{dates_csv_file}'")

# Close the browser
driver.quit()

# Merge files
merged_file = 'cartalk_topic_replies02.csv'

# Read the existing replies file
replies_data = []
with open(replies_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)  # Read the header row
    for row in reader:
        replies_data.append(row)

# Read the dates file and create a dictionary
dates_dict = {}
with open(dates_csv_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        topic_link = row[0]
        reply_date = row[1]
        if topic_link in dates_dict:
            dates_dict[topic_link].append(reply_date)
        else:
            dates_dict[topic_link] = [reply_date]

# Merge the data
merged_data = []
for row in replies_data:
    topic_link = row[0]
    reply = row[1]
    date = dates_dict.get(topic_link, [''])[0]  # Default to an empty string if the date doesn't exist
    merged_data.append([topic_link, reply, date])
    if topic_link in dates_dict:
        dates_dict[topic_link].pop(0)  # Remove the used date

# Save the merged data
with open(merged_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Topic Link', 'Reply', 'Date'])
    
    for data in merged_data:
        writer.writerow(data)

print(f"All data have been merged and saved to '{merged_file}'")



In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# Use webdriver_manager to automatically download and manage chromedriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Set explicit wait time to 30 seconds
replies_file = 'cartalk_topic_replies3.csv'

# Read the topic links from the existing replies file
topic_links = []
with open(replies_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        topic_links.append(row[0])

# Only process the first 2000 links (adjust as needed)
topic_links = topic_links[:2000]

# List to store the scraped dates
dates_data = []

for link in topic_links:
    # Open each topic link
    driver.get(link)
    
    # Wait for the page to load
    time.sleep(8)
    
    # Get the page content
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    
    # Find the dates
    dates = soup.find_all('span', class_='relative-date')  # Assuming dates are in a span with this class name
    
    for date in dates:
        reply_date = date['title'] if 'title' in date.attrs else date.get_text(strip=True)
        dates_data.append([link, reply_date])
    
    print(f"Processed {link}")

# Save the scraped dates to a new CSV file
dates_csv_file = 'cartalk_topic_dates3.csv'
with open(dates_csv_file, mode='w', newline='', encoding='utf-8') as file: 
    writer = csv.writer(file)
    writer.writerow(['Topic Link', 'Date'])
    
    for data in dates_data:
        writer.writerow(data)

print(f"All dates have been saved to '{dates_csv_file}'")

# Close the browser
driver.quit()

# Merge files
merged_file = 'cartalk_topic_replies03.csv'

# Read the existing replies file
replies_data = []
with open(replies_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)  # Read the header row
    for row in reader:
        replies_data.append(row)

# Read the dates file and create a dictionary
dates_dict = {}
with open(dates_csv_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        topic_link = row[0]
        reply_date = row[1]
        if topic_link in dates_dict:
            dates_dict[topic_link].append(reply_date)
        else:
            dates_dict[topic_link] = [reply_date]

# Merge the data
merged_data = []
for row in replies_data:
    topic_link = row[0]
    reply = row[1]
    date = dates_dict.get(topic_link, [''])[0]  # Default to an empty string if the date doesn't exist
    merged_data.append([topic_link, reply, date])
    if topic_link in dates_dict:
        dates_dict[topic_link].pop(0)  # Remove the used date

# Save the merged data
with open(merged_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Topic Link', 'Reply', 'Date'])
    
    for data in merged_data:
        writer.writerow(data)

print(f"All data have been merged and saved to '{merged_file}'")


In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# Use webdriver_manager to automatically download and manage chromedriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Set explicit wait time to 30 seconds
replies_file = 'cartalk_topic_replies4.csv'

# Read the Topic Links from the existing replies file
topic_links = []
with open(replies_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        topic_links.append(row[0])

# Only process the first 2000 links (adjust as needed)
topic_links = topic_links[:2000]

# List to store the scraped dates
dates_data = []

for link in topic_links:
    # Open each topic link
    driver.get(link)
    
    # Wait for the page to load
    time.sleep(8)
    
    # Get the page content
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    
    # Find the dates
    dates = soup.find_all('span', class_='relative-date')  # Assuming dates are in a span with this class name
    
    for date in dates:
        reply_date = date['title'] if 'title' in date.attrs else date.get_text(strip=True)
        dates_data.append([link, reply_date])
    
    print(f"Processed {link}")

# Save the scraped dates to a new CSV file
dates_csv_file = 'cartalk_topic_dates4.csv'
with open(dates_csv_file, mode='w', newline='', encoding='utf-8') as file: 
    writer = csv.writer(file)
    writer.writerow(['Topic Link', 'Date'])
    
    for data in dates_data:
        writer.writerow(data)

print(f"All dates have been saved to '{dates_csv_file}'")

# Close the browser
driver.quit()

# Merge files
merged_file = 'cartalk_topic_replies04.csv'

# Read the existing replies file
replies_data = []
with open(replies_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)  # Read the header row
    for row in reader:
        replies_data.append(row)

# Read the dates file and create a dictionary
dates_dict = {}
with open(dates_csv_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        topic_link = row[0]
        reply_date = row[1]
        if topic_link in dates_dict:
            dates_dict[topic_link].append(reply_date)
        else:
            dates_dict[topic_link] = [reply_date]

# Merge the data
merged_data = []
for row in replies_data:
    topic_link = row[0]
    reply = row[1]
    date = dates_dict.get(topic_link, [''])[0]  # Default to an empty string if the date doesn't exist
    merged_data.append([topic_link, reply, date])
    if topic_link in dates_dict:
        dates_dict[topic_link].pop(0)  # Remove the used date

# Save the merged data
with open(merged_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Topic Link', 'Reply', 'Date'])
    
    for data in merged_data:
        writer.writerow(data)

print(f"All data have been merged and saved to '{merged_file}'")


In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# Use webdriver_manager to automatically download and manage chromedriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Set explicit wait time to 30 seconds
replies_file = 'cartalk_topic_replies5.csv'

# Read the Topic Links from the existing replies file
topic_links = []
with open(replies_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        topic_links.append(row[0])

# Only process the first 2000 links (adjust as needed)
topic_links = topic_links[:2000]

# List to store the scraped dates
dates_data = []

for link in topic_links:
    # Open each topic link
    driver.get(link)
    
    # Wait for the page to load
    time.sleep(8)
    
    # Get the page content
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    
    # Find the dates
    dates = soup.find_all('span', class_='relative-date')  # Assuming dates are in a span with this class name
    
    for date in dates:
        reply_date = date['title'] if 'title' in date.attrs else date.get_text(strip=True)
        dates_data.append([link, reply_date])
    
    print(f"Processed {link}")

# Save the scraped dates to a new CSV file
dates_csv_file = 'cartalk_topic_dates5.csv'
with open(dates_csv_file, mode='w', newline='', encoding='utf-8') as file: 
    writer = csv.writer(file)
    writer.writerow(['Topic Link', 'Date'])
    
    for data in dates_data:
        writer.writerow(data)

print(f"All dates have been saved to '{dates_csv_file}'")

# Close the browser
driver.quit()

# Merge files
merged_file = 'cartalk_topic_replies05.csv'

# Read the existing replies file
replies_data = []
with open(replies_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)  # Read the header row
    for row in reader:
        replies_data.append(row)

# Read the dates file and create a dictionary
dates_dict = {}
with open(dates_csv_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        topic_link = row[0]
        reply_date = row[1]
        if topic_link in dates_dict:
            dates_dict[topic_link].append(reply_date)
        else:
            dates_dict[topic_link] = [reply_date]

# Merge the data
merged_data = []
for row in replies_data:
    topic_link = row[0]
    reply = row[1]
    date = dates_dict.get(topic_link, [''])[0]  # Default to an empty string if the date doesn't exist
    merged_data.append([topic_link, reply, date])
    if topic_link in dates_dict:
        dates_dict[topic_link].pop(0)  # Remove the used date

# Save the merged data
with open(merged_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Topic Link', 'Reply', 'Date'])
    
    for data in merged_data:
        writer.writerow(data)

print(f"All data have been merged and saved to '{merged_file}'")


In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# Use webdriver_manager to automatically download and manage chromedriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Set explicit wait time to 30 seconds
replies_file = 'cartalk_topic_replies6.csv'

# Read the Topic Links from the existing replies file
topic_links = []
with open(replies_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        topic_links.append(row[0])

# Only process the first 2000 links (adjust as needed)
topic_links = topic_links[:2000]

# List to store the scraped dates
dates_data = []

for link in topic_links:
    # Open each topic link
    driver.get(link)
    
    # Wait for the page to load
    time.sleep(8)
    
    # Get the page content
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    
    # Find the dates
    dates = soup.find_all('span', class_='relative-date')  # Assuming dates are in a span with this class name
    
    for date in dates:
        reply_date = date['title'] if 'title' in date.attrs else date.get_text(strip=True)
        dates_data.append([link, reply_date])
    
    print(f"Processed {link}")

# Save the scraped dates to a new CSV file
dates_csv_file = 'cartalk_topic_dates6.csv'
with open(dates_csv_file, mode='w', newline='', encoding='utf-8') as file: 
    writer = csv.writer(file)
    writer.writerow(['Topic Link', 'Date'])
    
    for data in dates_data:
        writer.writerow(data)

print(f"All dates have been saved to '{dates_csv_file}'")

# Close the browser
driver.quit()

# Merge files
merged_file = 'cartalk_topic_replies06.csv'

# Read the existing replies file
replies_data = []
with open(replies_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)  # Read the header row
    for row in reader:
        replies_data.append(row)

# Read the dates file and create a dictionary
dates_dict = {}
with open(dates_csv_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        topic_link = row[0]
        reply_date = row[1]
        if topic_link in dates_dict:
            dates_dict[topic_link].append(reply_date)
        else:
            dates_dict[topic_link] = [reply_date]

# Merge the data
merged_data = []
for row in replies_data:
    topic_link = row[0]
    reply = row[1]
    date = dates_dict.get(topic_link, [''])[0]  # Default to an empty string if the date doesn't exist
    merged_data.append([topic_link, reply, date])
    if topic_link in dates_dict:
        dates_dict[topic_link].pop(0)  # Remove the used date

# Save the merged data
with open(merged_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Topic Link', 'Reply', 'Date'])
    
    for data in merged_data:
        writer.writerow(data)

print(f"All data have been merged and saved to '{merged_file}'")


In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# Use webdriver_manager to automatically download and manage chromedriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Set explicit wait time to 30 seconds
replies_file = 'cartalk_topic_replies7.csv'

# Read the Topic Links from the existing replies file
topic_links = []
with open(replies_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        topic_links.append(row[0])

# Only process the first 2000 links (adjust as needed)
topic_links = topic_links[:2000]

# List to store the scraped dates
dates_data = []

for link in topic_links:
    # Open each topic link
    driver.get(link)
    
    # Wait for the page to load
    time.sleep(8)
    
    # Get the page content
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    
    # Find the dates
    dates = soup.find_all('span', class_='relative-date')  # Assuming dates are in a span with this class name
    
    for date in dates:
        reply_date = date['title'] if 'title' in date.attrs else date.get_text(strip=True)
        dates_data.append([link, reply_date])
    
    print(f"Processed {link}")

# Save the scraped dates to a new CSV file
dates_csv_file = 'cartalk_topic_dates7.csv'
with open(dates_csv_file, mode='w', newline='', encoding='utf-8') as file: 
    writer = csv.writer(file)
    writer.writerow(['Topic Link', 'Date'])
    
    for data in dates_data:
        writer.writerow(data)

print(f"All dates have been saved to '{dates_csv_file}'")

# Close the browser
driver.quit()

# Merge files
merged_file = 'cartalk_topic_replies07.csv'

# Read the existing replies file
replies_data = []
with open(replies_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)  # Read the header row
    for row in reader:
        replies_data.append(row)

# Read the dates file and create a dictionary
dates_dict = {}
with open(dates_csv_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        topic_link = row[0]
        reply_date = row[1]
        if topic_link in dates_dict:
            dates_dict[topic_link].append(reply_date)
        else:
            dates_dict[topic_link] = [reply_date]

# Merge the data
merged_data = []
for row in replies_data:
    topic_link = row[0]
    reply = row[1]
    date = dates_dict.get(topic_link, [''])[0]  # Default to an empty string if the date doesn't exist
    merged_data.append([topic_link, reply, date])
    if topic_link in dates_dict:
        dates_dict[topic_link].pop(0)  # Remove the used date

# Save the merged data
with open(merged_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Topic Link', 'Reply', 'Date'])
    
    for data in merged_data:
        writer.writerow(data)

print(f"All data have been merged and saved to '{merged_file}'")


In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time

# Use webdriver_manager to automatically download and manage chromedriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Set explicit wait time to 30 seconds
replies_file = 'cartalk_topic_replies8.csv'

# Read the Topic Links from the existing replies file
topic_links = []
with open(replies_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        topic_links.append(row[0])

# Only process the first 2000 links (adjust as needed)
topic_links = topic_links[:2000]

# List to store the scraped dates
dates_data = []

for link in topic_links:
    # Open each topic link
    driver.get(link)
    
    # Wait for the page to load
    time.sleep(8)
    
    # Get the page content
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    
    # Find the dates
    dates = soup.find_all('span', class_='relative-date')  # Assuming dates are in a span with this class name
    
    for date in dates:
        reply_date = date['title'] if 'title' in date.attrs else date.get_text(strip=True)
        dates_data.append([link, reply_date])
    
    print(f"Processed {link}")

# Save the scraped dates to a new CSV file
dates_csv_file = 'cartalk_topic_dates8.csv'
with open(dates_csv_file, mode='w', newline='', encoding='utf-8') as file: 
    writer = csv.writer(file)
    writer.writerow(['Topic Link', 'Date'])
    
    for data in dates_data:
        writer.writerow(data)

print(f"All dates have been saved to '{dates_csv_file}'")

# Close the browser
driver.quit()

# Merge files
merged_file = 'cartalk_topic_replies08.csv'

# Read the existing replies file
replies_data = []
with open(replies_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    header = next(reader)  # Read the header row
    for row in reader:
        replies_data.append(row)

# Read the dates file and create a dictionary
dates_dict = {}
with open(dates_csv_file, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        topic_link = row[0]
        reply_date = row[1]
        if topic_link in dates_dict:
            dates_dict[topic_link].append(reply_date)
        else:
            dates_dict[topic_link] = [reply_date]

# Merge the data
merged_data = []
for row in replies_data:
    topic_link = row[0]
    reply = row[1]
    date = dates_dict.get(topic_link, [''])[0]  # Default to an empty string if the date doesn't exist
    merged_data.append([topic_link, reply, date])
    if topic_link in dates_dict:
        dates_dict[topic_link].pop(0)  # Remove the used date

# Save the merged data
with open(merged_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Topic Link', 'Reply', 'Date'])
    
    for data in merged_data:
        writer.writerow(data)

print(f"All data have been merged and saved to '{merged_file}'")


In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# Use webdriver_manager to automatically download and manage chromedriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Set an explicit wait time of 30 seconds
wait = WebDriverWait(driver, 30)

# Read the topic links from the CSV file
topic_links = []
with open('cartalk_topic_links.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        topic_links.append(row[0])

# Process only the links from the 17,000th to the 19,000th
topic_links = topic_links[17000:19000]

# List to store the scraped replies
replies_data = []

for link in topic_links:
    # Open each topic link
    driver.get(link)
    
    # Wait for the page to load
    time.sleep(8)
    
    # Get the page content
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    
    # Find the reply content and dates
    replies = soup.find_all('div', class_='cooked')  # Assuming replies are in a div with this class name
    dates = soup.find_all('span', class_='relative-date')  # Assuming dates are in a span with this class name
    
    for reply, date in zip(replies, dates):
        reply_text = reply.get_text(strip=True)
        reply_date = date['title'] if 'title' in date.attrs else date.get_text(strip=True)
        replies_data.append([link, reply_text, reply_date])
    
    print(f"Processed {link}")

# Save the scraped replies to a new CSV file
replies_csv_file = 'cartalk_topic_replies9.csv'
with open(replies_csv_file, mode='w', newline='', encoding='utf-8') as file: 
    writer = csv.writer(file)
    writer.writerow(['Topic Link', 'Reply', 'Date'])
    
    for data in replies_data:
        writer.writerow(data)

print(f"All replies have been saved to '{replies_csv_file}'")

# Close the browser
driver.quit()


In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# Use webdriver_manager to automatically download and manage chromedriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Set an explicit wait time of 30 seconds
wait = WebDriverWait(driver, 30)

# Read topic links from the CSV file
topic_links = []
with open('cartalk_topic_links.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        topic_links.append(row[0])

# Process only links from index 19000 to 20000
topic_links = topic_links[19000:20000]

# List to store the scraped replies
replies_data = []

for link in topic_links:
    # Open each topic link
    driver.get(link)
    
    # Wait for the page to load
    time.sleep(8)
    
    # Get the page content
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    
    # Find reply content and date
    replies = soup.find_all('div', class_='cooked')  # Assuming reply content is in a div with this class name
    dates = soup.find_all('span', class_='relative-date')  # Assuming dates are in a span with this class name
    
    for reply, date in zip(replies, dates):
        reply_text = reply.get_text(strip=True)
        reply_date = date['title'] if 'title' in date.attrs else date.get_text(strip=True)
        replies_data.append([link, reply_text, reply_date])
    
    print(f"Processed {link}")

# Save the scraped replies to a new CSV file
replies_csv_file = 'cartalk_topic_replies10.csv'
with open(replies_csv_file, mode='w', newline='', encoding='utf-8') as file: 
    writer = csv.writer(file)
    writer.writerow(['Topic Link', 'Reply', 'Date'])
    
    for data in replies_data:
        writer.writerow(data)

print(f"All replies have been saved to '{replies_csv_file}'")

# Close the browser
driver.quit()


In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

# Use webdriver_manager to automatically download and manage chromedriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Set an explicit wait time of 30 seconds
wait = WebDriverWait(driver, 30)

# Read topic links from the CSV file
topic_links = []
with open('cartalk_topic_links.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip the header row
    for row in reader:
        topic_links.append(row[0])

# Process only links from index 20000 to 21000
topic_links = topic_links[20000:21000]

# List to store the scraped replies
replies_data = []

for link in topic_links:
    # Open each topic link
    driver.get(link)
    
    # Wait for the page to load
    time.sleep(8)
    
    # Get the page content
    page_content = driver.page_source
    soup = BeautifulSoup(page_content, 'html.parser')
    
    # Find reply content and date
    replies = soup.find_all('div', class_='cooked')  # Assuming reply content is in a div with this class name
    dates = soup.find_all('span', class_='relative-date')  # Assuming dates are in a span with this class name
    
    for reply, date in zip(replies, dates):
        reply_text = reply.get_text(strip=True)
        reply_date = date['title'] if 'title' in date.attrs else date.get_text(strip=True)
        replies_data.append([link, reply_text, reply_date])
    
    print(f"Processed {link}")

# Save the scraped replies to a new CSV file
replies_csv_file = 'cartalk_topic_replies11.csv'
with open(replies_csv_file, mode='w', newline='', encoding='utf-8') as file: 
    writer = csv.writer(file)
    writer.writerow(['Topic Link', 'Reply', 'Date'])
    
    for data in replies_data:
        writer.writerow(data)

print(f"All replies have been saved to '{replies_csv_file}'")

# Close the browser
driver.quit()

### 2. Data Labelling

In [ ]:
import pandas as pd

# Load the file
df1 = pd.read_excel('sample_for_manual_labeling.xlsx')

# Replace NaN values
df1['Reply'] = df1['Reply'].fillna('')

# Define the keywords for each category
electric_fit_keywords = ["battery", "charged", "charge", "charging", "software", "performance", "quality", "technical", "drive", "driving", "experience", "power", "torque", "autonomous", "range", "miles"]
affordability_keywords = ["cost", "price", "pricing", "expensive", "cheap", "maintenance", "repair", "expenses", "charge cost", "£", "$", "€", "money", "value"]
customer_care_keywords = ["dealer", "dealership", "purchase", "buy", "after-sales", "support", "service", "customer service", "satisfaction", "repair", "warranty", "contact", "helpful", "unhelpful"]

# Function to label the replies
def label_reply_multiple(reply, keywords):
    reply = reply.lower() 
    return any(keyword in reply for keyword in keywords)

# Apply the labeling function for each category
df1['Electric Fit'] = df1['Reply'].apply(lambda x: 1 if label_reply_multiple(x, electric_fit_keywords) else 0)
df1['Affordability'] = df1['Reply'].apply(lambda x: 1 if label_reply_multiple(x, affordability_keywords) else 0)
df1['Customer Care'] = df1['Reply'].apply(lambda x: 1 if label_reply_multiple(x, customer_care_keywords) else 0)

# Save the labeled data to a new Excel file
output_file_path = 'multi_labelled.xlsx'
df1.to_excel(output_file_path, index=False)

output_file_path

### 3. Name Recognition

The version I've tried before discuss with group members.

In [ ]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import pandas as pd

# Load SpaCy model
nlp = spacy.load('en_core_web_sm')

# Register and add the TextBlob component
spacy_text_blob = SpacyTextBlob(nlp)
nlp.add_pipe("spacytextblob", last=True)

# Load the dataset
df_new = pd.read_csv('new_cleaned_filtered_main_removed_missing_duplicates_with_predictions.csv')

# Perform sentiment analysis using SpaCy and TextBlob
if 'Review' in df_new.columns:
    def get_sentiment(text):
        doc = nlp(text)
        return doc._.polarity, doc._.subjectivity

    df_new['Polarity'], df_new['Subjectivity'] = zip(*df_new['Review'].apply(get_sentiment))

# Keywords to filter and count
keywords = ['mini', 'nissan', 'vauxhall', 'honda', 'peugeot', 'renault', 'zoe', 'leaf', 'bmw', 'i3', 'hyundai', 'kona', 'chevrolet', 'bolt', 'fiat', '500e', 'cooper se']

# Function to recognize car brands and ensure case-insensitive matching
def recognize_brand(text, brands):
    text_lower = text.lower()
    for brand in brands:
        if brand.lower() in text_lower:
            return brand
    return None

# Apply the function to each review to find the car brand
df_new['Car Brand'] = df_new['Review'].apply(lambda x: recognize_brand(x, keywords))

# Function to filter reviews mentioning specified keywords and count occurrences
def count_reviews_with_keywords(df, keywords):
    keywords_lower = [keyword.lower() for keyword in keywords]  # Ensure all keywords are lowercase
    def contains_keyword(review):
        review_lower = str(review).lower()
        return any(keyword in review_lower for keyword in keywords_lower)
    
    filtered_df = df[df['Review'].apply(contains_keyword)]
    counts = filtered_df['Review'].str.lower().apply(lambda x: [keyword for keyword in keywords_lower if keyword in x]).explode().value_counts()
    
    return filtered_df, counts

# Get the filtered reviews and counts with the combined approach
filtered_reviews, keyword_counts = count_reviews_with_keywords(df_new, keywords)

# Convert keyword counts to DataFrame for better visibility
keyword_counts_df = keyword_counts.reset_index()
keyword_counts_df.columns = ['Keyword', 'Count']

# Save the updated DataFrame to a new CSV file
df_new.to_csv('updated_reviews_with_sentiment_and_car_brand_combined.csv', index=False)

# Print the top 5 most common car brands and all recognized brands
print("Top 5 most common car brands:\n", keyword_counts_df.head())
print("\nFrequency of all recognized brands:\n", keyword_counts_df)

In [ ]:
df= pd.read_csv('new_cleaned_filtered_main_removed_missing_duplicates_with_predictions.csv')
# Dictionary mapping models to their respective brands
model_to_brand = {
    'mini': 'Mini', 'cooper se': 'Mini',
    'nissan': 'Nissan', 'leaf': 'Nissan',
    'vauxhall': 'Vauxhall', 'corsa': 'Vauxhall',
    'honda': 'Honda',
    'peugeot': 'Peugeot', '208': 'Peugeot', 'e-208': 'Peugeot',
    'renault': 'Renault', 'zoe': 'Renault',
    'fiat': 'Fiat', '500e': 'Fiat',
    'bmw': 'BMW', 'i3': 'BMW',
    'hyundai': 'Hyundai', 'kona': 'Hyundai',
    'chevrolet': 'Chevrolet', 'bolt': 'Chevrolet'
}

# Improved function to include variations in keyword case and relate brands and models
def count_reviews_with_keywords(df, model_to_brand):
    keywords_lower = {k.lower(): v for k, v in model_to_brand.items()}  # Lowercase keywords and map to brand
    def contains_keyword(review):
        review_lower = str(review).lower()
        return any(keyword in review_lower for keyword in keywords_lower)
    
    filtered_df = df[df['Review'].apply(contains_keyword)]
    counts = filtered_df['Review'].str.lower().apply(lambda x: [keywords_lower[keyword] for keyword in keywords_lower if keyword in x]).explode().value_counts()
    
    return filtered_df, counts

# Get the filtered reviews and counts with improved function
filtered_reviews, keyword_counts = count_reviews_with_keywords(df_new, model_to_brand)

# Convert keyword counts to DataFrame for better visibility
keyword_counts_df = keyword_counts.reset_index()
keyword_counts_df.columns = ['Brand', 'Count']

# Display the results
filtered_reviews.head(), keyword_counts_df

final version of Name recognition part!

In [ ]:
# read data
df_new=pd.read_csv('new_cleaned_filtered_main_removed_missing_duplicates_with_predictions.csv')

#### 3.1 Brand Mapping

In [ ]:
# Dictionary mapping models to their respective brands
model_to_brand = {
    'mini': 'Mini', 'cooper se': 'Mini',
    'nissan': 'Nissan', 'leaf': 'Nissan',
    'vauxhall': 'Vauxhall', 'corsa': 'Vauxhall',
    'honda': 'Honda','honda e': 'Honda',
    'peugeot': 'Peugeot', '208': 'Peugeot', 'e-208': 'Peugeot',
    'renault': 'Renault', 'zoe': 'Renault',
    'fiat': 'Fiat', '500e': 'Fiat',
    'bmw': 'BMW', 'i3': 'BMW',
    'hyundai': 'Hyundai', 'kona': 'Hyundai',
    'chevrolet': 'Chevrolet', 'bolt': 'Chevrolet'
}

# Improved function to include variations in keyword case and relate brands and models
def count_reviews_with_keywords(df, model_to_brand):
    keywords_lower = {k.lower(): v for k, v in model_to_brand.items()}  # Lowercase keywords and map to brand
    def contains_keyword(review):
        review_lower = str(review).lower()
        return any(keyword in review_lower for keyword in keywords_lower)
    
    filtered_df = df[df['Review'].apply(contains_keyword)]
    counts = filtered_df['Review'].str.lower().apply(lambda x: [keywords_lower[keyword] for keyword in keywords_lower if keyword in x]).explode().value_counts()
    
    return filtered_df, counts

# Get the filtered reviews and counts with improved function
filtered_reviews, keyword_counts = count_reviews_with_keywords(df_new, model_to_brand)

# Convert keyword counts to DataFrame for better visibility
keyword_counts_df = keyword_counts.reset_index()
keyword_counts_df.columns = ['Brand', 'Count']

# Display the results
filtered_reviews.head(), keyword_counts_df

#### 3.2 Filter Fiat and Competitors(Mini, Honda, Vauxhall, Peugeot)

In [ ]:
# Dictionary mapping models to their respective brands
model_to_brand = {
    'mini': 'Mini', 'cooper se': 'Mini',
    'nissan': 'Nissan', 'leaf': 'Nissan',
    'vauxhall': 'Vauxhall', 'corsa': 'Vauxhall',
    'honda': 'Honda', 'honda e': 'Honda',
    'peugeot': 'Peugeot', '208': 'Peugeot', 'e-208': 'Peugeot',
    'renault': 'Renault', 'zoe': 'Renault',
    'fiat': 'Fiat', '500e': 'Fiat',
    'bmw': 'BMW', 'i3': 'BMW',
    'hyundai': 'Hyundai', 'kona': 'Hyundai',
    'chevrolet': 'Chevrolet', 'bolt': 'Chevrolet'
}

# Function to label reviews with brands
def label_reviews_with_brands(df, model_to_brand):
    keywords_lower = {k.lower(): v for k, v in model_to_brand.items()}  # Lowercase keywords and map to brand
    
    def find_brand(review):
        review_lower = str(review).lower()
        for keyword, brand in keywords_lower.items():
            if keyword in review_lower:
                return brand
        return None
    
    df['Brand'] = df['Review'].apply(find_brand)
    
    # Filter for only the desired brands
    desired_brands = ['Mini', 'Honda', 'Vauxhall', 'Peugeot','Fiat']
    df_filtered = df[df['Brand'].isin(desired_brands)]
    
    return df_filtered

# Apply the function to the dataframe
df_filtered = label_reviews_with_brands(df_new, model_to_brand)


#### 3.3 Final Dataset
Remove the unclear date, and assume it as our final dataset

In [ ]:
# Load the new file for analysis
new_cleaned_filtered_df = pd.read_csv('finaldf_V2.csv')

# Display unique values in the 'Date' column to identify any non-date entries
unique_dates = new_cleaned_filtered_df['Date'].unique()
unique_dates

In [ ]:
# Load the dataframe and convert the 'Date' column to strings
new_cleaned_filtered_df = pd.read_csv('finaldf_V2.csv', dtype={'Author': str})
new_cleaned_filtered_df['Date'] = new_cleaned_filtered_df['Date'].astype(str)

# Identify rows with non-date entries in the 'Date' column
non_date_rows = new_cleaned_filtered_df[~new_cleaned_filtered_df['Date'].str.match(r'^\d{4}/\d{2}/\d{2}$')]

# Remove these rows from the main dataframe
cleaned_df = new_cleaned_filtered_df[new_cleaned_filtered_df['Date'].str.match(r'^\d{4}/\d{2}/\d{2}$')]

# Save the non-date rows to a new file
non_date_rows.to_csv('non_date_entries1.csv', index=False)

# Save the cleaned dataframe to a new file
cleaned_df.to_csv('final.csv', index=False)

# Display the non-date rows
print(non_date_rows)

In [ ]:
# Check the data header
# Load the cleaned dataset
cleaned_df = pd.read_csv('final.csv')

# Ensure the 'Date' column is in datetime format
cleaned_df['Date'] = pd.to_datetime(cleaned_df['Date'])

# Display the first few rows to verify
cleaned_df.head()

## 4. Sentiment Analysis

In [ ]:
# load dataset
import pandas as pd

df=pd.read_csv('final.csv')
df.head()

#### 4.1 Sentiment Scores

In [ ]:
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import seaborn as sns

sia = SentimentIntensityAnalyzer()

# Function to analyze sentiment
def analyze_sentiment(text):
    return sia.polarity_scores(text)['compound']

# Apply sentiment analysis on each review
df['Sentiment_Score'] = df['Cleaned_Review'].apply(analyze_sentiment)

# Define the labels of interest as a dictionary with their keywords
labels_of_interest = {
    'Electric Fit': 'Electric Fit',
    'Affordability': 'Affordability',
    'Customer Care': 'Customer Care'
}

# Initialize an empty DataFrame to store the results
df_filtered = pd.DataFrame()

# Iterate over each category and filter the DataFrame
for category, keyword in labels_of_interest.items():
    # Filter rows where the Predicted_Label contains the keyword
    df_category = df[df['Predicted_Label'].str.contains(keyword, case=False, na=False)].copy()
    # Assign the category to a new column, adding to any existing categories
    if 'Category' in df_category.columns:
        df_category['Category'] = df_category['Category'] + ', ' + category
    else:
        df_category['Category'] = category
    # Append the results to the final DataFrame
    df_filtered = pd.concat([df_filtered, df_category])

# Group by brand and category, then calculate mean sentiment score
sentiment_by_category = df_filtered.groupby(['Brand', 'Category'])['Sentiment_Score'].mean().unstack()

# Visualization
plt.figure(figsize=(12, 8))
sns.heatmap(sentiment_by_category, annot=True, cmap='coolwarm')
plt.title('Sentiment Analysis by Brand and Category')
plt.show()

#### 4.2 Sentiment Distribution

In [ ]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import pandas as pd

# Load SpaCy model
nlp = spacy.load('en_core_web_sm')

# Register and add the TextBlob component
spacy_text_blob = SpacyTextBlob(nlp)
nlp.add_pipe("spacytextblob", last=True)

# Function to process reviews based on a specific label and perform sentiment analysis
def process_reviews_by_label(df, label, output_filename):
    # Filter reviews where the label is present in the 'Predicted_Label'
    filtered_reviews = df[df['Predicted_Label'].apply(lambda x: label in x)]
    
    # Perform Sentiment Analysis using Cleaned_Review column with SpaCy and spacytextblob
    def get_sentiment(text):
        doc = nlp(text)
        if doc._.polarity > 0:
            return 'positive'
        elif doc._.polarity < 0:
            return 'negative'
        else:
            return 'neutral'

    filtered_reviews['Sentiment'] = filtered_reviews['Cleaned_Review'].apply(get_sentiment)

    # Aggregate results by car brand and sentiment
    sentiment_summary = filtered_reviews.groupby(['Brand', 'Sentiment']).size().unstack(fill_value=0)

    # Add a 'Total' column to summarize the number of reviews for each sentiment category
    sentiment_summary['Total'] = sentiment_summary.sum(axis=1)
    
    # Save the filtered reviews with sentiment analysis to a new CSV file
    filtered_reviews.to_csv(output_filename, index=False)

    # Display the path to the saved dataset
    print(f"Dataset with sentiments saved to: {output_filename}")
    return sentiment_summary

# Load the dataset
df = pd.read_csv('final.csv')

# Process reviews for different labels
labels = ['Electric Fit', 'Affordability', 'Customer Care']
output_files = ['electric_fit.csv', 'affordability.csv', 'customer_care.csv']

for label, output_file in zip(labels, output_files):
    sentiment_summary = process_reviews_by_label(df, label, output_file)
    print(f"\nSentiment summary for {label}:")
    print(sentiment_summary)

#### 4.3 Visualise in Count

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

# Load the datasets
electric_fit_df = pd.read_csv('electric_fit.csv')
customer_care_df = pd.read_csv('customer_care.csv')
affordability_df = pd.read_csv('affordability.csv')

# Function to visualize sentiment distribution
def visualize_sentiment_distribution(df, kpi_name):
    sentiment_summary = df.groupby(['Brand', 'Sentiment']).size().unstack(fill_value=0)
    sentiment_summary.plot(kind='bar', stacked=True, figsize=(10, 6), title=f'Sentiment Distribution for {kpi_name}')
    plt.xlabel('Brand')
    plt.ylabel('Count')
    plt.legend(title='Sentiment')
    plt.show()

# Visualize each KPI
visualize_sentiment_distribution(electric_fit_df, 'Electric Fit')
visualize_sentiment_distribution(customer_care_df, 'Customer Care')
visualize_sentiment_distribution(affordability_df, 'Affordability')

#### 4.4 Visualise in Percentage

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

# Load the datasets
electric_fit_df = pd.read_csv('electric_fit.csv')
customer_care_df = pd.read_csv('customer_care.csv')
affordability_df = pd.read_csv('affordability.csv')

# Function to visualize sentiment distribution as percentages
def visualize_sentiment_distribution(df, kpi_name):
    sentiment_summary = df.groupby(['Brand', 'Sentiment']).size().unstack(fill_value=0)
    sentiment_percentage = sentiment_summary.div(sentiment_summary.sum(axis=1), axis=0) * 100
    sentiment_percentage.plot(kind='bar', stacked=True, figsize=(10, 6), title=f'Sentiment Distribution for {kpi_name} (Percentage)')
    plt.xlabel('Brand')
    plt.ylabel('Percentage')
    plt.legend(title='Sentiment')
    plt.show()

# Visualize each KPI as percentage
visualize_sentiment_distribution(electric_fit_df, 'Electric Fit')
visualize_sentiment_distribution(customer_care_df, 'Customer Care')
visualize_sentiment_distribution(affordability_df, 'Affordability')


### 5. Topic Modelling

#### 5.1 Included others

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from gensim.models import LdaModel
from gensim.corpora import Dictionary  # Import the Dictionary class
import pyLDAvis.gensim_models as gensimvis
from pyLDAvis import display

# Function to split KPIs from the Predicted_Label column
def split_kpis(predicted_label):
    return predicted_label.strip("()").replace("'", "").split(", ")

# Apply the split function to create a KPI column
df['KPI'] = df['Predicted_Label'].apply(split_kpis)

# Explode the KPI column to have one KPI per row
df_exploded = df.explode('KPI')

# Now you can split the data into Fiat and competitors
fiat_df = df_exploded[df_exploded['Brand'] == 'Fiat']
competitors_df = df_exploded[df_exploded['Brand'] != 'Fiat']

print(f"Fiat dataset contains {len(fiat_df)} reviews.")
print(f"Competitors dataset contains {len(competitors_df)} reviews.")

In [ ]:
# For Fiat
fiat_reviews = fiat_df['Cleaned_Review'].tolist()
fiat_texts = [review.split() for review in fiat_reviews]  # Tokenize the cleaned reviews
fiat_id2word = Dictionary(fiat_texts)
fiat_corpus = [fiat_id2word.doc2bow(text) for text in fiat_texts]

# For Competitors
competitors_reviews = competitors_df['Cleaned_Review'].tolist()
competitors_texts = [review.split() for review in competitors_reviews]  # Tokenize the cleaned reviews
competitors_id2word = Dictionary(competitors_texts)
competitors_corpus = [competitors_id2word.doc2bow(text) for text in competitors_texts]

print('Fiat corpus contains {} reviews with {} unique words'.format(len(fiat_corpus), len(fiat_id2word)))
print('Competitors corpus contains {} reviews with {} unique words'.format(len(competitors_corpus), len(competitors_id2word)))


In [ ]:
# Number of topics
num_topics = 15

# Train LDA model for Fiat
fiat_lda_model = LdaModel(fiat_corpus, num_topics=num_topics, id2word=fiat_id2word, passes=15)
print("Fiat LDA Model:")
for idx, topic in fiat_lda_model.print_topics(num_topics=num_topics, num_words=10):
    print(f"Topic {idx}: {topic}")

# Train LDA model for Competitors
competitors_lda_model = LdaModel(competitors_corpus, num_topics=num_topics, id2word=competitors_id2word, passes=15)
print("\nCompetitors LDA Model:")
for idx, topic in competitors_lda_model.print_topics(num_topics=num_topics, num_words=10):
    print(f"Topic {idx}: {topic}")

In [ ]:
# Visualize Fiat LDA Model
fiat_lda_vis = gensimvis.prepare(fiat_lda_model, fiat_corpus, fiat_id2word)
display(fiat_lda_vis)

In [ ]:
# Visualize Competitors LDA Model
competitors_lda_vis = gensimvis.prepare(competitors_lda_model, competitors_corpus, competitors_id2word)
display(competitors_lda_vis)

In [ ]:
# Function to extract topics
def extract_topics(reviews, model, id2word, num_topics):
    topics = []
    for review in reviews:
        bow = id2word.doc2bow(review.split())
        doc_topics = model.get_document_topics(bow)
        topic_vector = [0] * num_topics
        for topic_num, score in doc_topics:
            topic_vector[topic_num] = score
        topics.append(topic_vector)
    return topics

# Extract topics for Fiat
fiat_df['Topics'] = extract_topics(fiat_reviews, fiat_lda_model, fiat_id2word, num_topics)

# Extract topics for Competitors
competitors_df['Topics'] = extract_topics(competitors_reviews, competitors_lda_model, competitors_id2word, num_topics)

# Convert 'Topics' column to a DataFrame with each topic as a separate column
fiat_topics_df = pd.DataFrame(fiat_df['Topics'].tolist(), columns=['Fiat_Topic_' + str(i) for i in range(num_topics)])
competitors_topics_df = pd.DataFrame(competitors_df['Topics'].tolist(), columns=['Competitors_Topic_' + str(i) for i in range(num_topics)])

# Concatenate the topics DataFrame with the original DataFrame
fiat_df = pd.concat([fiat_df.reset_index(drop=True), fiat_topics_df], axis=1)
competitors_df = pd.concat([competitors_df.reset_index(drop=True), competitors_topics_df], axis=1)

# Calculate average topic distributions for Fiat and competitors
# We only want to calculate the mean for the topic columns, which are numeric
fiat_topic_columns = [col for col in fiat_df.columns if col.startswith('Fiat_Topic_')]
competitors_topic_columns = [col for col in competitors_df.columns if col.startswith('Competitors_Topic_')]

fiat_averages = fiat_df[fiat_topic_columns].mean()
competitors_averages = competitors_df[competitors_topic_columns].mean()

print(f"Fiat Average Topic Distributions:\n{fiat_averages}")
print(f"Competitors Average Topic Distributions:\n{competitors_averages}")

####  5.2 Exclude Others

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from gensim.models import LdaModel
from gensim.corpora import Dictionary
import pyLDAvis.gensim_models as gensimvis
from pyLDAvis import display

# Function to split KPIs from the Predicted_Label column
def split_kpis(predicted_label):
    return predicted_label.strip("()").replace("'", "").split(", ")

# Apply the split function to create a KPI column
df['KPI'] = df['Predicted_Label'].apply(split_kpis)

# Explode the KPI column to have one KPI per row
df_exploded = df.explode('KPI')

# Exclude the "Other" label from analysis
df_exploded = df_exploded[df_exploded['KPI'] != 'Other']

# Split the data into Fiat and competitors
fiat_df = df_exploded[df_exploded['Brand'] == 'Fiat']
competitors_df = df_exploded[df_exploded['Brand'] != 'Fiat']

print(f"Fiat dataset contains {len(fiat_df)} reviews.")
print(f"Competitors dataset contains {len(competitors_df)} reviews.")

# Build the dataset (corpus) for Fiat
fiat_reviews = fiat_df['Cleaned_Review'].tolist()
fiat_texts = [review.split() for review in fiat_reviews]  # Tokenize the cleaned reviews
fiat_id2word = Dictionary(fiat_texts)
fiat_corpus = [fiat_id2word.doc2bow(text) for text in fiat_texts]
print('Fiat corpus contains {} reviews with {} unique words'.format(len(fiat_corpus), len(fiat_id2word)))

# Train the topic model for Fiat
num_topics = 15  # Number of topics
fiat_lda_model = LdaModel(fiat_corpus, num_topics=num_topics, id2word=fiat_id2word, passes=15)

# Print all topics for Fiat
print("Fiat LDA Model:")
for idx, topic in fiat_lda_model.print_topics(num_topics=num_topics, num_words=10):
    print(f"Topic {idx}: {topic}")

# Build the dataset (corpus) for Competitors
competitors_reviews = competitors_df['Cleaned_Review'].tolist()
competitors_texts = [review.split() for review in competitors_reviews]  # Tokenize the cleaned reviews
competitors_id2word = Dictionary(competitors_texts)
competitors_corpus = [competitors_id2word.doc2bow(text) for text in competitors_texts]
print('Competitors corpus contains {} reviews with {} unique words'.format(len(competitors_corpus), len(competitors_id2word)))

# Train the topic model for Competitors
competitors_lda_model = LdaModel(competitors_corpus, num_topics=num_topics, id2word=competitors_id2word, passes=15)

# Print all topics for Competitors
print("\nCompetitors LDA Model:")
for idx, topic in competitors_lda_model.print_topics(num_topics=num_topics, num_words=10):
    print(f"Topic {idx}: {topic}")


In [ ]:
# Visualize the topics using pyLDAvis for Fiat
fiat_lda_vis = gensimvis.prepare(fiat_lda_model, fiat_corpus, fiat_id2word)
display(fiat_lda_vis)

In [ ]:
# Visualize the topics using pyLDAvis for Competitors
competitors_lda_vis = gensimvis.prepare(competitors_lda_model, competitors_corpus, competitors_id2word)
display(competitors_lda_vis)

In [ ]:
# Function to extract topics from reviews
def extract_topics(reviews, model, id2word, num_topics):
    topics = []
    for review in reviews:
        bow = id2word.doc2bow(review.split())
        doc_topics = model.get_document_topics(bow)
        topic_vector = [0] * num_topics
        for topic_num, score in doc_topics:
            topic_vector[topic_num] = score
        topics.append(topic_vector)
    return topics

# Extract topics for Fiat
fiat_df['Topics'] = extract_topics(fiat_reviews, fiat_lda_model, fiat_id2word, num_topics)

# Extract topics for Competitors
competitors_df['Topics'] = extract_topics(competitors_reviews, competitors_lda_model, competitors_id2word, num_topics)

# Convert 'Topics' column to a DataFrame with each topic as a separate column for Fiat
fiat_topics_df = pd.DataFrame(fiat_df['Topics'].tolist(), columns=['Fiat_Topic_' + str(i) for i in range(num_topics)])

# Convert 'Topics' column to a DataFrame with each topic as a separate column for Competitors
competitors_topics_df = pd.DataFrame(competitors_df['Topics'].tolist(), columns=['Competitors_Topic_' + str(i) for i in range(num_topics)])

# Concatenate the topics DataFrame with the original DataFrame for Fiat
fiat_df = pd.concat([fiat_df.reset_index(drop=True), fiat_topics_df], axis=1)

# Concatenate the topics DataFrame with the original DataFrame for Competitors
competitors_df = pd.concat([competitors_df.reset_index(drop=True), competitors_topics_df], axis=1)

# Ensure that the topic columns are numeric
fiat_topic_columns = ['Fiat_Topic_' + str(i) for i in range(num_topics)]
competitors_topic_columns = ['Competitors_Topic_' + str(i) for i in range(num_topics)]

fiat_df[fiat_topic_columns] = fiat_df[fiat_topic_columns].apply(pd.to_numeric)
competitors_df[competitors_topic_columns] = competitors_df[competitors_topic_columns].apply(pd.to_numeric)

# Calculate average topic distributions for Fiat and Competitors
fiat_averages = fiat_df[fiat_topic_columns].mean()
competitors_averages = competitors_df[competitors_topic_columns].mean()

print(f"Fiat Average Topic Distributions:\n{fiat_averages}")
print(f"Competitors Average Topic Distributions:\n{competitors_averages}")

###  6. Time-series

I've try the basic time-series at first, Due to the distribution of tasks, our group member take over it so that i didnt extend the analysis.

In [ ]:
import pandas as pd

# Load the cleaned dataset
cleaned_df = pd.read_csv('final.csv')
cleaned_df['Date'] = pd.to_datetime(cleaned_df['Date'], errors='coerce')

# Load the sentiment data files
ef_df = pd.read_csv('electric_fit_filtered_reviews.csv')
af_df = pd.read_csv('affordability_filtered_reviews.csv')
ccf_df = pd.read_csv('customer_care_filtered_reviews.csv')

# Convert Date columns to datetime
ef_df['Date'] = pd.to_datetime(ef_df['Date'], errors='coerce')
af_df['Date'] = pd.to_datetime(af_df['Date'], errors='coerce')
ccf_df['Date'] = pd.to_datetime(ccf_df['Date'], errors='coerce')

In [ ]:
import matplotlib.pyplot as plt

# Function to plot sentiment trends
def plot_sentiment_trends(df, title):
    sentiment_trends = df.groupby(['Date', 'Sentiment']).size().unstack(fill_value=0)
    sentiment_trends.plot(figsize=(14, 7))
    plt.title(f'{title} Sentiment Trends Over Time')
    plt.xlabel('Date')
    plt.ylabel('Number of Reviews')
    plt.legend(title='Sentiment')
    plt.grid(True)
    plt.show()

# Plot sentiment trends for Electric Fit
plot_sentiment_trends(ef_df, 'Electric Fit')

# Plot sentiment trends for Affordability
plot_sentiment_trends(af_df, 'Affordability')

# Plot sentiment trends for Customer Care
plot_sentiment_trends(ccf_df, 'Customer Care')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Function to plot smoothed sentiment trends
def plot_smoothed_sentiment_trends(df, title, window=30):
    sentiment_trends = df.groupby(['Date', 'Sentiment']).size().unstack(fill_value=0)
    sentiment_trends = sentiment_trends.rolling(window=window).mean()
    
    sentiment_trends.plot(figsize=(14, 7))
    plt.title(f'{title} Sentiment Trends Over Time (Smoothed)')
    plt.xlabel('Date')
    plt.ylabel('Number of Reviews')
    plt.legend(title='Sentiment')
    plt.grid(True)
    plt.show()

# Plot smoothed sentiment trends for Electric Fit
plot_smoothed_sentiment_trends(ef_df, 'Electric Fit')

# Plot smoothed sentiment trends for Affordability
plot_smoothed_sentiment_trends(af_df, 'Affordability')

# Plot smoothed sentiment trends for Customer Care
plot_smoothed_sentiment_trends(ccf_df, 'Customer Care')

In [ ]:
# Function to plot sentiment trends separately
def plot_separate_sentiment_trends(df, title):
    sentiment_trends = df.groupby(['Date', 'Sentiment']).size().unstack(fill_value=0)
    
    fig, axes = plt.subplots(3, 1, figsize=(14, 14), sharex=True)
    sentiments = ['positive', 'neutral', 'negative']
    colors = ['green', 'orange', 'red']
    
    for sentiment, color, ax in zip(sentiments, colors, axes):
        sentiment_trend = sentiment_trends[sentiment].rolling(window=30).mean()
        ax.plot(sentiment_trend.index, sentiment_trend, label=sentiment, color=color)
        ax.set_title(f'{title} - {sentiment.capitalize()} Sentiment')
        ax.set_ylabel('Number of Reviews')
        ax.legend()
        ax.grid(True)
    
    plt.xlabel('Date')
    plt.show()

# Plot separate sentiment trends for Electric Fit
plot_separate_sentiment_trends(ef_df, 'Electric Fit')

# Plot separate sentiment trends for Affordability
plot_separate_sentiment_trends(af_df, 'Affordability')

# Plot separate sentiment trends for Customer Care
plot_separate_sentiment_trends(ccf_df, 'Customer Care')

In [ ]:
# Function to plot stacked area sentiment trends
def plot_stacked_area_sentiment_trends(df, title):
    sentiment_trends = df.groupby(['Date', 'Sentiment']).size().unstack(fill_value=0)
    sentiment_trends = sentiment_trends.rolling(window=30).mean()
    
    sentiment_trends.plot.area(figsize=(14, 7), stacked=True)
    plt.title(f'{title} Sentiment Trends Over Time (Stacked Area)')
    plt.xlabel('Date')
    plt.ylabel('Number of Reviews')
    plt.legend(title='Sentiment')
    plt.grid(True)
    plt.show()

# Plot stacked area sentiment trends for Electric Fit
plot_stacked_area_sentiment_trends(ef_df, 'Electric Fit')

# Plot stacked area sentiment trends for Affordability
plot_stacked_area_sentiment_trends(af_df, 'Affordability')

# Plot stacked area sentiment trends for Customer Care
plot_stacked_area_sentiment_trends(ccf_df, 'Customer Care')